In [3]:
import sys
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis\read")

import numpy
import math
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import calibrate_power as cbp

from scipy import signal, optimize, ndimage
from read_scope_data import read_trc_data, read_txt_data

%matplotlib widget

In [3]:
ifn = r"C:\data\epfl\interferometer\birdcage\C1--800W-5mT--00000.trc"
phase_raw, tarr_ne = read_trc_data(ifn, True)

ifn = r"C:\data\epfl\interferometer\birdcage\C2--800W-5mT--00000.trc"
amp_raw, tarr_ne = read_trc_data(ifn)

print(tarr_ne.shape)

dt = 8.000000106811456e-10
t0 = -1.0004389373219374e-06
vertical_gain = 200 mV per div
timebase = 200 ns per div
Input =  DC 1 MOhm
Reading data...
Done
Reading data...
Done
(100002,)


In [5]:
step_a = 1000
step_b = 20000
step_c = -20000
step_d = -1000

fig = plt.figure()
plt.plot(tarr_ne, phase_raw)
plt.plot(tarr_ne[step_a:step_b], phase_raw[step_a:step_b])
plt.plot(tarr_ne[step_c:step_d], phase_raw[step_c:step_d])

delV = numpy.average(phase_raw[step_c:step_d]) - numpy.average(phase_raw[step_a:step_b])
print("voltage difference = ", delV)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

voltage difference =  0.6378672680966434


In [7]:
plt.close(fig)

phase = (delV) / 1.8 * numpy.pi / (2*numpy.pi) # 1.8V => pi; in units of 2pi (number of fringes)
ne = phase/1.34e-16 * 100 / 0.3 # f0=100GHz; path length 30cm; assume uniform density profile

print(ne)

4.3416150547122627e+18
6.909894969596462e+17


In [ ]:
ifn = r"C:\data\epfl\CRDS\scope trc\solAnt\C3-500W-15mT-00000.trc"
light, tarr = read_trc_data(ifn)
print(tarr.shape, light.shape)

ifn = r"C:\data\epfl\CRDS\solenoid\20230601_15mTorr_0.15lnmin.xlsx"

# xlsx is only supported by openpyxl special package because they saved the excel on a very old version ugh
excel_data_df = pd.read_excel(ifn, sheet_name=2, engine='openpyxl', header=[0])

hdr_ls = excel_data_df.columns.ravel()

tarr_crds = excel_data_df[hdr_ls[0]].to_numpy()
sig_crds = excel_data_df[hdr_ls[1]].to_numpy()

tarr_crds = tarr_crds[~numpy.isnan(sig_crds)]
sig_crds = sig_crds[~numpy.isnan(sig_crds)]

In [ ]:
#-----------normalized ne-------------------------
ne_arr = (numpy.max(phase_raw) - phase_raw) / numpy.max(phase_raw)
ne_arr /= numpy.max(ne_arr)

plt.figure()
plt.plot((tarr_ne-tarr_ne[60000])*1e3, ne_arr) #74500

#-----------normalized light-------------------------
light_norm = light / numpy.max(light)

plt.plot(tarr*1e3, light_norm)

#-----------normalized CRDS-------------------------
tarr_fake = tarr_crds - tarr_crds[-551]
sig_crds_mod = (numpy.roll(sig_crds,-550) - numpy.average(sig_crds[90:100]))
sig_crds_mod /= numpy.max(sig_crds_mod)

plt.plot(tarr_fake*1e3, sig_crds_mod)